In [ ]:
import yaml
import zarr

repo_root = "/Users/miskodzamba/Dropbox/research/gits/spf/"
import sys

sys.path.append(repo_root)  # go to parent dir

root = "/Users/miskodzamba/Dropbox/research/spf_aux/missions/march27_wall"
prefix = "wallarrayv3_2024_03_27_01_11_08_nRX2_rx_circle"
zarr_fn = f"{root}/{prefix}.zarr"
yaml_fn = f"{root}/{prefix}.yaml"
z = zarr.open(zarr_fn, "r")

yaml_config = yaml.safe_load(open(yaml_fn, "r"))

In [ ]:
# self.steering_vectors = precompute_steering_vectors(
#     receiver_positions=self.rx_config.rx_pos,
#     carrier_frequency=self.rx_config.lo,
#     spacing=self.nthetas,
# )

from spf.data_collector import rx_config_from_receiver_yaml
from spf.rf import beamformer_given_steering, precompute_steering_vectors
from spf.sdrpluto.sdr_controller import get_avg_phase
import numpy as np
from matplotlib import pyplot as plt
from spf.rf import beamformer_given_steering_nomean

nthetas = 64 + 1

rx_configs = [
    rx_config_from_receiver_yaml(receiver) for receiver in yaml_config["receivers"]
]

steering_vectors = [
    precompute_steering_vectors(
        receiver_positions=rx_config.rx_pos,
        carrier_frequency=rx_config.lo,
        spacing=nthetas,
    )
    for rx_config in rx_configs
]


def load_zarr_to_numpy(x):
    return np.array(x)


session_idx = 10
beam_sds = [
    beamformer_given_steering_nomean(
        steering_vectors=steering_vectors[receiver_idx],
        signal_matrix=load_zarr_to_numpy(
            z.receivers[f"r{receiver_idx}"].signal_matrix[session_idx]
        ),
    )
    for receiver_idx in range(2)
]

# avg_phase_diff = get_avg_phase(signal_matrix)

In [ ]:
from spf.sdrpluto.sdr_controller import get_phase_diff

session_idx = 10
signal_matrix = load_zarr_to_numpy(z.receivers["r0"].signal_matrix[session_idx])
n = 2 * 4 * 50000
pd = get_phase_diff(signal_matrix[:, :n])
v = signal_matrix[0, :n]
fig, axs = plt.subplots(2, 1, figsize=(12, 4))
axs[0].scatter(np.arange(n), v.real, alpha=0.1, s=1)
axs[1].scatter(np.arange(n), pd, s=1, alpha=0.1)

In [ ]:
z.tree()

In [ ]:
z.receivers.r0.signal_matrix[0].shape